# Mentor Feedback Implementation — GIM Mess Food-Waste Forecasting

This notebook adds the exact items requested by your mentor **without altering your baseline**:
1. Rolling-origin (expanding window) backtests vs seasonal-naive; report **MAE/SMAPE** (overall and by month).
2. **Decision layer** with prediction intervals via quantile regression (0.1/0.5/0.9); **interval coverage**.
3. **Event effects** modeling + **fallback for unseen events** via hierarchical pooling; error analysis by event type.
4. **Explainability**: SHAP (if installed) or permutation importance → actionable kitchen levers.
5. **Imputation** + **cold-start** strategy (weekday-grouped impute + hierarchical pooling).

Artifacts are saved to `artifacts/` and `reports/figures/`.


In [ ]:
# ## 0. Setup
import os, json, math, warnings
from pathlib import Path
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance

# Optional libs
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

RNG = np.random.RandomState(42)

ROOT = Path.cwd()
ARTIFACTS = ROOT / "artifacts"
FIGS = ROOT / "reports" / "figures"
ARTIFACTS.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

print("Artifacts:", ARTIFACTS)
print("Figures:", FIGS)


In [ ]:
# ## 1. Load Data and Identify Columns

# Try common locations; edit if needed
CANDIDATES = [
    ROOT / "data" / "mess_waste_GIM_500.csv",
    ROOT / "mess_waste_GIM_500.csv",
    ROOT / "data" / "mess_waste_GIM_daily_exams.csv",
]
def pick(paths):
    for p in paths:
        if p.exists():
            return p
    raise FileNotFoundError("Please update data path")

DATA_PATH = pick(CANDIDATES)
print("Using data:", DATA_PATH)

df = pd.read_csv(DATA_PATH)

# Infer date, target, event columns
DATE_CAND = ["date", "day", "timestamp", "dt"]
TARGET_CAND = ["food_waste_kg", "waste_kg", "leftover_kg", "wastage"]
EVENT_CAND = ["event", "event_type", "special_event"]

date_col = next((c for c in DATE_CAND if c in df.columns), None)
target = next((c for c in TARGET_CAND if c in df.columns), None)
event_col = next((c for c in EVENT_CAND if c in df.columns), None)

if date_col is None: 
    raise KeyError("No date column found. Add one of: " + ",".join(DATE_CAND))
if target is None:
    raise KeyError("No target column found. Add one of: " + ",".join(TARGET_CAND))

df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
df = df.sort_values(date_col).reset_index(drop=True)

# Basic engineered time features
df["dow"] = df[date_col].dt.dayofweek
df["month"] = df[date_col].dt.month
df["is_weekend"] = (df["dow"] >= 5).astype(int)

# Weekday-grouped imputation example (for attendance/menu counts etc.)
# Fill numeric NaNs with weekday means then global mean
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != target]
for c in num_cols:
    if df[c].isna().any():
        df[c] = df.groupby("dow")[c].transform(lambda s: s.fillna(s.mean()))
        df[c] = df[c].fillna(df[c].mean())

# Categorical impute: fill missing with 'Unknown'
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna("Unknown")
    
print("Columns:", df.columns.tolist())
print("Target:", target, "| Date:", date_col, "| Event:", event_col)


In [ ]:
# ## 2. Metrics, Baselines, and Utilities

def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    # Avoid division by zero
    mask = denom == 0
    out = np.zeros_like(denom)
    out[~mask] = diff[~mask] / denom[~mask]
    return 100.0 * np.mean(out)

def seasonal_naive(y_series, period=7):
    # y_hat[t] = y[t - period]
    y = np.asarray(y_series)
    y_hat = np.empty_like(y, dtype=float)
    y_hat[:] = np.nan
    for t in range(period, len(y)):
        y_hat[t] = y[t - period]
    return y_hat

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def month_name(dt):
    return dt.strftime("%Y-%m")

def interval_coverage(y_true, lo, hi):
    y = np.asarray(y_true)
    return np.mean((y >= lo) & (y <= hi))

def waste_shortage(prepared, actual):
    prepared = np.asarray(prepared)
    actual = np.asarray(actual)
    waste = np.maximum(0.0, prepared - actual)
    shortage = np.maximum(0.0, actual - prepared)
    return waste, shortage


In [ ]:
# ## 3. Preprocessor and Feature Matrix

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

# Do not leak target
if target in numeric_cols: numeric_cols.remove(target)

# Do not feed raw date; already encoded via features
if date_col in numeric_cols: numeric_cols.remove(date_col)
if date_col in categorical_cols: categorical_cols.remove(date_col)

preprocessor = ColumnTransformer([
    ("num", StandardScaler(with_mean=False), numeric_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
])
X = df.drop(columns=[target])
y = df[target].astype(float).values


In [ ]:
# ## 4. Rolling-Origin Backtest (Expanding Window, H=1)

# Models
rf = RandomForestRegressor(
    n_estimators=400, max_depth=None, random_state=42, n_jobs=-1
)

# Pipelines
from sklearn.pipeline import Pipeline
rf_pipe = Pipeline([("prep", preprocessor), ("rf", rf)])

# Seasonal naive requires only y history
y_naive = seasonal_naive(y, period=7)

# Rolling-origin expanding window
preds_rf = np.full_like(y, fill_value=np.nan, dtype=float)
months = df[date_col].dt.to_period("M").astype(str).values

start = 14  # minimal warmup (2 weeks) — adjust if needed
for t in range(start, len(df)-1):
    train_idx = slice(0, t+1)   # up to current t
    test_idx = t+1              # predict next day
    
    rf_pipe.fit(X.iloc[train_idx], y[train_idx])
    preds_rf[test_idx] = rf_pipe.predict(X.iloc[[test_idx]])[0]

# Metrics overall (align indices where both preds exist)
mask = ~np.isnan(preds_rf) & ~np.isnan(y_naive)
res_overall = {
    "RF_MAE": mae(y[mask], preds_rf[mask]),
    "RF_SMAPE": smape(y[mask], preds_rf[mask]),
    "Naive_MAE": mae(y[mask], y_naive[mask]),
    "Naive_SMAPE": smape(y[mask], y_naive[mask]),
    "Improvement_MAE_%": 100.0 * (1 - mae(y[mask], preds_rf[mask]) / mae(y[mask], y_naive[mask])),
    "Improvement_SMAPE_%": 100.0 * (1 - smape(y[mask], preds_rf[mask]) / smape(y[mask], y_naive[mask])),
}
print("Overall backtest:", res_overall)

# By month
df_bt = pd.DataFrame({
    "month": months,
    "y": y,
    "rf": preds_rf,
    "naive": y_naive
})
df_bt = df_bt.dropna()

monthly = []
for m, g in df_bt.groupby("month"):
    monthly.append({
        "month": m,
        "RF_MAE": mae(g["y"], g["rf"]),
        "RF_SMAPE": smape(g["y"], g["rf"]),
        "Naive_MAE": mae(g["y"], g["naive"]),
        "Naive_SMAPE": smape(g["y"], g["naive"]),
    })
monthly_df = pd.DataFrame(monthly).sort_values("month")

# Save
pd.DataFrame([res_overall]).to_csv(ARTIFACTS / "backtest_overall.csv", index=False)
monthly_df.to_csv(ARTIFACTS / "backtest_monthly.csv", index=False)
print("Saved:", ARTIFACTS / "backtest_overall.csv")
print("Saved:", ARTIFACTS / "backtest_monthly.csv")


In [ ]:
# ## 5. Quantile Regression Decision Layer (0.1, 0.5, 0.9)

# We'll use GradientBoostingRegressor with quantile loss to avoid extra dependencies.
q_models = {}
for q in [0.1, 0.5, 0.9]:
    q_models[q] = Pipeline([
        ("prep", preprocessor),
        ("gbr", GradientBoostingRegressor(loss="quantile", alpha=q, random_state=42, n_estimators=400, max_depth=3))
    ])

q_lo = np.full_like(y, np.nan, dtype=float)
q_md = np.full_like(y, np.nan, dtype=float)
q_hi = np.full_like(y, np.nan, dtype=float)

for t in range(start, len(df)-1):
    train_idx = slice(0, t+1)
    test_idx = t+1
    for q in [0.1, 0.5, 0.9]:
        q_models[q].fit(X.iloc[train_idx], y[train_idx])
    q_lo[test_idx] = q_models[0.1].predict(X.iloc[[test_idx]])[0]
    q_md[test_idx] = q_models[0.5].predict(X.iloc[[test_idx]])[0]
    q_hi[test_idx] = q_models[0.9].predict(X.iloc[[test_idx]])[0]

mask_q = ~np.isnan(q_lo) & ~np.isnan(q_md) & ~np.isnan(q_hi)
coverage90 = interval_coverage(y[mask_q], q_lo[mask_q], q_hi[mask_q])
print("90% interval coverage:", coverage90)

# Policy simulation: prepared = q_md + lambda*(q_hi - q_md)
lambdas = np.linspace(0, 1, 6)
policy_rows = []
for lam in lambdas:
    prepared = q_md.copy()
    prepared[mask_q] = q_md[mask_q] + lam * (q_hi[mask_q] - q_md[mask_q])
    w, s = waste_shortage(prepared[mask_q], y[mask_q])
    policy_rows.append({
        "lambda": lam,
        "avg_waste": np.mean(w),
        "avg_shortage": np.mean(s),
        "efficiency_%": 100.0 * (1 - (np.mean(w) / (np.mean(y[mask_q]) + 1e-8)))  # heuristic
    })
policy_df = pd.DataFrame(policy_rows)
policy_df.to_csv(ARTIFACTS / "policy_simulation.csv", index=False)
print("Saved:", ARTIFACTS / "policy_simulation.csv")

# Save interval coverage
with open(ARTIFACTS / "interval_coverage.json", "w") as f:
    json.dump({"coverage90": float(coverage90)}, f, indent=2)
print("Saved:", ARTIFACTS / "interval_coverage.json")


In [ ]:
# ## 6. Event Effects Modeling + Hierarchical Pooling Fallback

# We assume event_col is categorical like ['Normal','Fest','Exam', ...].
# We'll compute residuals by event type from the RF rolling forecast,
# and implement a shrinkage estimate for unseen events.

if event_col is None:
    print("No explicit event column found; synthesizing 'Normal' as placeholder.")
    df["event_synth"] = "Normal"
    event_col_use = "event_synth"
else:
    event_col_use = event_col

df_bt2 = pd.DataFrame({
    "date": df[date_col],
    "y": y,
    "rf": preds_rf,
    "event": df[event_col_use]
}).dropna()

df_bt2["resid"] = df_bt2["y"] - df_bt2["rf"]

# Per-event residual summaries
evt_stats = df_bt2.groupby("event")["resid"].agg(["mean","std","count"]).reset_index().rename(columns={"mean":"resid_mean","std":"resid_std","count":"n"})
evt_stats["global_mean"] = df_bt2["resid"].mean()
evt_stats["global_var"] = df_bt2["resid"].var()

# Empirical Bayes style shrinkage: m = (n/(n+k))*event_mean + (k/(n+k))*global_mean
k = max(1.0, evt_stats["n"].median())  # pseudo-count
evt_stats["shrink_mean"] = (evt_stats["n"]/(evt_stats["n"]+k))*evt_stats["resid_mean"] + (k/(evt_stats["n"]+k))*evt_stats["global_mean"]

evt_stats.to_csv(ARTIFACTS / "event_residuals.csv", index=False)
print("Saved:", ARTIFACTS / "event_residuals.csv")


In [ ]:
# ## 7. Explainability → Actionable Levers (SHAP if available else permutation)

# Fit final RF on full data to extract importances
rf_final = Pipeline([("prep", preprocessor), ("rf", RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1))])
rf_final.fit(X, y)

feat_names = []
try:
    # numeric first
    feat_names += list(preprocessor.transformers_[0][2])
    # then onehot names
    ohe = preprocessor.named_transformers_["cat"]
    cat_base = preprocessor.transformers_[1][2]
    ohe_names = ohe.get_feature_names_out(cat_base).tolist()
    feat_names += ohe_names
except Exception:
    feat_names = [f"f{i}" for i in range(rf_final.named_steps["rf"].n_features_in_)]

importance_df = None
if SHAP_AVAILABLE:
    try:
        explainer = shap.TreeExplainer(rf_final.named_steps["rf"])
        X_trans = preprocessor.fit_transform(X)  # ensure fitted names align
        # Use a small sample for speed
        idx = np.linspace(0, X_trans.shape[0]-1, min(500, X_trans.shape[0])).astype(int)
        shap_values = explainer.shap_values(X_trans[idx])
        shap_abs_mean = np.mean(np.abs(shap_values), axis=0)
        importance_df = pd.DataFrame({"feature": feat_names, "importance": shap_abs_mean}).sort_values("importance", ascending=False)
    except Exception as e:
        print("SHAP failed, falling back to permutation:", e)

if importance_df is None:
    perm = permutation_importance(rf_final, X, y, n_repeats=5, random_state=42, n_jobs=-1)
    importance_df = pd.DataFrame({"feature": feat_names, "importance": perm.importances_mean}).sort_values("importance", ascending=False)

importance_df.to_csv(ARTIFACTS / "feature_importance.csv", index=False)
print("Saved:", ARTIFACTS / "feature_importance.csv")


In [ ]:
# ## 8. Error Analysis: Top Mispredictions with Root Causes

# Use RF rolling predictions (preds_rf) for realistic errors
bt_err = pd.DataFrame({
    "date": df[date_col],
    "y": y,
    "rf": preds_rf
})
bt_err["abs_err"] = np.abs(bt_err["y"] - bt_err["rf"])
bt_err = bt_err.dropna().sort_values("abs_err", ascending=False)

TOPK = 15
context_cols = [c for c in df.columns if c not in [target]]
top_err = bt_err.head(TOPK).merge(df[context_cols], left_index=True, right_index=True, how="left")
top_err.to_csv(ARTIFACTS / "top_mispredictions.csv", index=False)
print("Saved:", ARTIFACTS / "top_mispredictions.csv")


In [ ]:
# ## 9. Quick Plots (saved for Overleaf)

import matplotlib.pyplot as plt

# Monthly MAE/SMAPE bar
fig1 = plt.figure(figsize=(8,4))
plt.plot(monthly_df["month"], monthly_df["RF_MAE"], marker='o', label="RF MAE")
plt.plot(monthly_df["month"], monthly_df["Naive_MAE"], marker='o', label="Naive MAE")
plt.xticks(rotation=45, ha='right')
plt.title("Monthly MAE — Rolling-Origin Backtest")
plt.legend()
plt.tight_layout()
p1 = FIGS / "monthly_mae.png"
plt.savefig(p1, dpi=150)
plt.show()
print("Saved:", p1)

fig2 = plt.figure(figsize=(8,4))
plt.plot(monthly_df["month"], monthly_df["RF_SMAPE"], marker='o', label="RF SMAPE")
plt.plot(monthly_df["month"], monthly_df["Naive_SMAPE"], marker='o', label="Naive SMAPE")
plt.xticks(rotation=45, ha='right')
plt.title("Monthly SMAPE — Rolling-Origin Backtest")
plt.legend()
plt.tight_layout()
p2 = FIGS / "monthly_smape.png"
plt.savefig(p2, dpi=150)
plt.show()
print("Saved:", p2)

# Quantile interval band (last 60 days if available)
fig3 = plt.figure(figsize=(9,4))
mask_plot = mask_q.copy()
# focus last window
idxs = np.where(mask_plot)[0]
if len(idxs) > 0:
    tail = idxs[-min(60, len(idxs)):]  # last 60 predictions
    plt.fill_between(tail, q_lo[tail], q_hi[tail], alpha=0.3, label="90% PI")
    plt.plot(tail, y[tail], label="Actual")
    plt.plot(tail, q_md[tail], label="Median forecast")
    plt.title("Quantile Intervals (last window)")
    plt.legend()
    plt.tight_layout()
    p3 = FIGS / "quantile_intervals.png"
    plt.savefig(p3, dpi=150)
    plt.show()
    print("Saved:", p3)

# Feature importance top-20
top_imp = importance_df.head(20).iloc[::-1]
fig4 = plt.figure(figsize=(7,6))
plt.barh(top_imp["feature"], top_imp["importance"])
plt.title("Top Feature Importances")
plt.tight_layout()
p4 = FIGS / "feature_importance.png"
plt.savefig(p4, dpi=150)
plt.show()
print("Saved:", p4)


In [ ]:
# ## 10. Outputs Index

outputs = {
    "overall_backtest_csv": str(ARTIFACTS / "backtest_overall.csv"),
    "monthly_backtest_csv": str(ARTIFACTS / "backtest_monthly.csv"),
    "interval_coverage_json": str(ARTIFACTS / "interval_coverage.json"),
    "policy_simulation_csv": str(ARTIFACTS / "policy_simulation.csv"),
    "event_residuals_csv": str(ARTIFACTS / "event_residuals.csv"),
    "feature_importance_csv": str(ARTIFACTS / "feature_importance.csv"),
    "top_mispredictions_csv": str(ARTIFACTS / "top_mispredictions.csv"),
    "fig_monthly_mae": str(FIGS / "monthly_mae.png"),
    "fig_monthly_smape": str(FIGS / "monthly_smape.png"),
    "fig_quantile_intervals": str(FIGS / "quantile_intervals.png"),
    "fig_feature_importance": str(FIGS / "feature_importance.png"),
}
print(json.dumps(outputs, indent=2))
